In [11]:
from __future__ import print_function

import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

In [12]:
pickle_file= 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save=pickle.load(f)
    train_dataset=save['train_dataset']
    train_labels=save['train_labels']
    valid_dataset=save['valid_dataset']
    valid_labels=save['valid_labels']
    test_dataset=save['test_dataset']
    test_labels=save['test_labels']
    del save #hint to help gc to free up memory 
    print('training set', train_dataset.shape,train_labels.shape)
    print('validation set', valid_dataset.shape, valid_labels.shape)
    print('test set', test_dataset.shape,test_labels.shape)

training set (200000, 28, 28) (200000,)
validation set (10000, 28, 28) (10000,)
test set (10000, 28, 28) (10000,)


In [13]:
image_size=28
num_labels=10
num_channels=1 

def reformat(dataset, labels):
    dataset=dataset.reshape((-1,image_size,image_size,num_channels)).astype(np.float32)
    # Map 1 to [0.0,1.0,0.0....], 2 to [0.0,0.0,1.0.....]
    labels=(np.arange(num_labels) ==labels[:,None]).astype(np.float32)
    return dataset,labels
train_dataset, train_labels= reformat(train_dataset, train_labels)
valid_dataset, valid_labels=reformat(valid_dataset, valid_labels)
test_dataset, test_labels =reformat(test_dataset, test_labels)
print( 'training set', train_dataset.shape,train_labels.shape)
print('validation set', valid_dataset.shape,valid_labels.shape)
print('test set', test_dataset.shape,test_labels.shape)

training set (200000, 28, 28, 1) (200000, 10)
validation set (10000, 28, 28, 1) (10000, 10)
test set (10000, 28, 28, 1) (10000, 10)


In [14]:
def accuracy(predictions, labels):
    return(100.0*np.sum(np.argmax(predictions, 1)==np.argmax(labels,1))/ predictions.shape[0])

In [16]:
batch_size=16
patch_size=5
depth=16
num_hidden=64

graph=tf.Graph()

with graph.as_default():
    
    #input data
    tf_train_dataset=tf.placeholder(tf.float32, 
                                    shape=(batch_size,image_size,image_size,num_channels))
    tf_train_labels=tf.placeholder(tf.float32, shape=(batch_size,num_labels))
    tf_valid_dataset=tf.constant(valid_dataset)
    tf_test_dataset=tf.constant(test_dataset)
    
    #variables
    layer1_weights=tf.Variable(tf.truncated_normal
                               ([patch_size,patch_size,num_channels,depth],stddev=0.1))
    layer1_biases=tf.Variable(tf.zeros([depth]))
    
    layer2_weights=tf.Variable(tf.truncated_normal
                               ([patch_size,patch_size,depth,depth],stddev=0.1))
    layer2_biases=tf.Variable(tf.constant(1.0,shape=[depth]))
    
    
    layer3_weights=tf.Variable(tf.truncated_normal
                               ([image_size//4*image_size//4*depth,num_hidden],stddev=0.1))
    layer3_biases=tf.Variable(tf.constant(1.0,shape=[num_hidden]))
    
    
    layer4_weights=tf.Variable(tf.truncated_normal
                               ([num_hidden,num_labels],stddev=0.1))
    layer4_biases=tf.Variable(tf.constant(1.0,shape=[num_labels]))
    
    #Model
    def model(data):
        conv=tf.nn.conv2d(data,layer1_weights,[1,2,2,1], padding='SAME')
        hidden=tf.nn.relu(conv+layer1_biases)
        conv=tf.nn.conv2d(hidden,layer2_weights,[1,2,2,1], padding='SAME')
        hidden=tf.nn.relu(conv+layer2_biases)
        shape=hidden.get_shape().as_list()
        reshape=tf.reshape(hidden,[shape[0],shape[1]*shape[2]*shape[3]])
        hidden=tf.nn.relu(tf.matmul(reshape, layer3_weights)+layer3_biases)
        return tf.matmul(hidden, layer4_weights)+ layer4_biases
    
    #training computation
    
    logits=model(tf_train_dataset)
    loss=tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels,logits=logits))
    
    #optimizer
    
    optimizer=tf.train.GradientDescentOptimizer(0.05).minimize(loss)
    
    #Predictions for the training , validation, and test data
    
    train_prediction= tf.nn.softmax(logits)
    valid_prediction= tf.nn.softmax(model(tf_valid_dataset))
    test_prediction= tf.nn.softmax(model(tf_test_dataset))

In [20]:
num_steps=10001

with tf.Session(graph=graph)as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        offset=(step*batch_size)% (train_labels.shape[0]-batch_size)
        batch_data=train_dataset[offset:(offset+batch_size),:,:,:]
        batch_labels=train_labels[offset:(offset+batch_size),:]
        feed_dict={tf_train_dataset : batch_data,tf_train_labels: batch_labels}
        _, l, predictions =session.run([optimizer,loss,train_prediction],feed_dict=feed_dict)
        if(step%50==0):
            print('Minibatch loss a step %d: %f' %(step,l))
            print('Minibatch Accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%'%accuracy(valid_prediction.eval(),valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))
    

Initialized
Minibatch loss a step 0: 3.263589
Minibatch Accuracy: 12.5%
Validation accuracy: 11.4%
Minibatch loss a step 50: 1.235159
Minibatch Accuracy: 56.2%
Validation accuracy: 50.5%
Minibatch loss a step 100: 1.398365
Minibatch Accuracy: 62.5%
Validation accuracy: 72.7%
Minibatch loss a step 150: 0.792629
Minibatch Accuracy: 75.0%
Validation accuracy: 76.2%
Minibatch loss a step 200: 0.529968
Minibatch Accuracy: 81.2%
Validation accuracy: 79.1%
Minibatch loss a step 250: 0.953043
Minibatch Accuracy: 75.0%
Validation accuracy: 79.8%
Minibatch loss a step 300: 0.644519
Minibatch Accuracy: 75.0%
Validation accuracy: 80.4%
Minibatch loss a step 350: 0.293357
Minibatch Accuracy: 87.5%
Validation accuracy: 79.4%
Minibatch loss a step 400: 0.484874
Minibatch Accuracy: 87.5%
Validation accuracy: 77.2%
Minibatch loss a step 450: 1.102701
Minibatch Accuracy: 62.5%
Validation accuracy: 80.3%
Minibatch loss a step 500: 0.256470
Minibatch Accuracy: 93.8%
Validation accuracy: 80.7%
Minibatch lo

# convulation using maxpooling operation(nn.max_pool()) of stride of 2 and kernel size 2.

In [21]:
batch_size=16
patch_size=5
depth=16
num_hidden=64

graph=tf.Graph()

with graph.as_default():
    
    #input data
    tf_train_dataset=tf.placeholder(tf.float32, 
                                    shape=(batch_size,image_size,image_size,num_channels))
    tf_train_labels=tf.placeholder(tf.float32, shape=(batch_size,num_labels))
    tf_valid_dataset=tf.constant(valid_dataset)
    tf_test_dataset=tf.constant(test_dataset)
    
    #variables
    layer1_weights=tf.Variable(tf.truncated_normal
                               ([patch_size,patch_size,num_channels,depth],stddev=0.1))
    layer1_biases=tf.Variable(tf.zeros([depth]))
    
    layer2_weights=tf.Variable(tf.truncated_normal
                               ([patch_size,patch_size,depth,depth],stddev=0.1))
    layer2_biases=tf.Variable(tf.constant(1.0,shape=[depth]))
    
    
    layer3_weights=tf.Variable(tf.truncated_normal
                               ([image_size//4*image_size//4*depth,num_hidden],stddev=0.1))
    layer3_biases=tf.Variable(tf.constant(1.0,shape=[num_hidden]))
    
    
    layer4_weights=tf.Variable(tf.truncated_normal
                               ([num_hidden,num_labels],stddev=0.1))
    layer4_biases=tf.Variable(tf.constant(1.0,shape=[num_labels]))
    
    #Model
    def model(data):
        #1st conv layer with pooling
        conv_1=tf.nn.conv2d(data,layer1_weights,[1,1,1,1], padding='SAME')
        hidden_1=tf.nn.relu(conv_1+layer1_biases)
        pool_1=tf.nn.max_pool(hidden_1,[1,2,2,1],[1,2,2,1], padding='SAME')
        
         #2nd conv layer with pooling
        conv_2=tf.nn.conv2d(pool_1,layer2_weights,[1,1,1,1], padding='SAME')
        hidden_2=tf.nn.relu(conv_2+layer2_biases)
        pool_2=tf.nn.max_pool(hidden_2,[1,2,2,1],[1,2,2,1], padding='SAME')
        
         #fully connected layer
        shape=pool_2.get_shape().as_list()
        reshape=tf.reshape(pool_2,[shape[0],shape[1]*shape[2]*shape[3]])
        hidden=tf.nn.relu(tf.matmul(reshape, layer3_weights)+layer3_biases)
        return tf.matmul(hidden, layer4_weights)+ layer4_biases
    
    #training computation
    
    logits=model(tf_train_dataset)
    loss=tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels,logits=logits))
    
    #optimizer
    
    optimizer=tf.train.GradientDescentOptimizer(0.05).minimize(loss)
    
    #Predictions for the training , validation, and test data
    
    train_prediction= tf.nn.softmax(logits)
    valid_prediction= tf.nn.softmax(model(tf_valid_dataset))
    test_prediction= tf.nn.softmax(model(tf_test_dataset))

In [22]:
num_steps=10001

with tf.Session(graph=graph)as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        offset=(step*batch_size)% (train_labels.shape[0]-batch_size)
        batch_data=train_dataset[offset:(offset+batch_size),:,:,:]
        batch_labels=train_labels[offset:(offset+batch_size),:]
        feed_dict={tf_train_dataset : batch_data,tf_train_labels: batch_labels}
        _, l, predictions =session.run([optimizer,loss,train_prediction],feed_dict=feed_dict)
        if(step%50==0):
            print('Minibatch loss a step %d: %f' %(step,l))
            print('Minibatch Accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%'%accuracy(valid_prediction.eval(),valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))
    

Initialized
Minibatch loss a step 0: 3.003009
Minibatch Accuracy: 6.2%
Validation accuracy: 10.0%
Minibatch loss a step 50: 2.212623
Minibatch Accuracy: 18.8%
Validation accuracy: 22.5%
Minibatch loss a step 100: 1.873110
Minibatch Accuracy: 56.2%
Validation accuracy: 64.0%
Minibatch loss a step 150: 1.151364
Minibatch Accuracy: 62.5%
Validation accuracy: 74.0%
Minibatch loss a step 200: 0.864053
Minibatch Accuracy: 75.0%
Validation accuracy: 76.9%
Minibatch loss a step 250: 0.878512
Minibatch Accuracy: 81.2%
Validation accuracy: 80.1%
Minibatch loss a step 300: 0.861306
Minibatch Accuracy: 81.2%
Validation accuracy: 79.8%
Minibatch loss a step 350: 0.286721
Minibatch Accuracy: 87.5%
Validation accuracy: 80.1%
Minibatch loss a step 400: 0.684663
Minibatch Accuracy: 87.5%
Validation accuracy: 78.6%
Minibatch loss a step 450: 1.018541
Minibatch Accuracy: 68.8%
Validation accuracy: 80.2%
Minibatch loss a step 500: 0.296469
Minibatch Accuracy: 93.8%
Validation accuracy: 81.6%
Minibatch los

# conv with dropout

In [26]:
image_size=28
#create image size function based on input, filter size, padding and stride
#2 convolutions only with 2 pooling
def output_size_pool(input_size, conv_filter_size,pool_filter_size, padding, conv_stride, pool_stride):
    if padding =='same':
        padding=-1.00
    elif padding=='valid':
        padding=0.00
    else:
        return None
    #after convolution 1
    output_1=(((input_size-conv_filter_size-2*padding)/ conv_stride)+1.00)
    #after pool 1
    output_2=(((output_1-pool_filter_size-2*padding)/pool_stride)+1.00)
    #After convolution 2
    output_3=(((output_2-conv_filter_size-2*padding)/conv_stride)+1.00)
    #after pool 2
    output_4=(((output_3-pool_filter_size-2*padding)/pool_stride)+1.00)
    return int(output_4)
final_image_size=output_size_pool(
    input_size=image_size,conv_filter_size=5,pool_filter_size=2,
    padding='valid', conv_stride=1, pool_stride=2)
print(final_image_size)
    

4


In [28]:
batch_size=16
patch_size=5
depth=32
num_hidden=64
beta=0.001

graph=tf.Graph()

with graph.as_default():
    
    #input data
    tf_train_dataset=tf.placeholder(tf.float32, 
                                    shape=(batch_size,image_size,image_size,num_channels))
    tf_train_labels=tf.placeholder(tf.float32, shape=(batch_size,num_labels))
    tf_valid_dataset=tf.constant(valid_dataset)
    tf_test_dataset=tf.constant(test_dataset)
    
    #variables
    layer1_weights=tf.Variable(tf.truncated_normal
                               ([patch_size,patch_size,num_channels,depth],stddev=0.1))
    layer1_biases=tf.Variable(tf.zeros([depth]))
    
    layer2_weights=tf.Variable(tf.truncated_normal
                               ([patch_size,patch_size,depth,depth],stddev=0.1))
    layer2_biases=tf.Variable(tf.constant(1.0,shape=[depth]))
    
    final_image_size=output_size_pool(input_size=image_size,
                                      conv_filter_size=5,pool_filter_size=2,padding='valid',
                                      conv_stride=1,pool_stride=2)
    
    
    layer3_weights=tf.Variable(tf.truncated_normal
                               ([final_image_size*final_image_size*depth,num_hidden],stddev=0.1))
    layer3_biases=tf.Variable(tf.constant(1.0,shape=[num_hidden]))
    
    
    layer4_weights=tf.Variable(tf.truncated_normal
                               ([num_hidden,num_hidden],stddev=0.1))
    layer4_biases=tf.Variable(tf.constant(1.0,shape=[num_hidden]))
    
    
    
    layer5_weights=tf.Variable(tf.truncated_normal
                               ([num_hidden,num_labels],stddev=0.1))
    layer5_biases=tf.Variable(tf.constant(1.0,shape=[num_labels]))
    
    #Model
    def model(data):
        #1st conv layer with pooling
        conv_1=tf.nn.conv2d(data,layer1_weights,[1,1,1,1], padding='VALID')
        hidden_1=tf.nn.relu(conv_1+layer1_biases)
        pool_1=tf.nn.avg_pool(hidden_1,[1,2,2,1],[1,2,2,1], padding='VALID')
        
         #2nd conv layer with pooling
        conv_2=tf.nn.conv2d(pool_1,layer2_weights,[1,1,1,1], padding='VALID')
        hidden_2=tf.nn.relu(conv_2+layer2_biases)
        pool_2=tf.nn.avg_pool(hidden_2,[1,2,2,1],[1,2,2,1], padding='VALID')
        
         #1st fully connected layer
        shape=pool_2.get_shape().as_list()
        reshape=tf.reshape(pool_2,[shape[0],shape[1]*shape[2]*shape[3]])
        hidden=tf.nn.relu(tf.matmul(reshape, layer3_weights)+layer3_biases)
        keep_prob=0.5
        hidden_drop=tf.nn.dropout(hidden,keep_prob)
        
        #2nd fully connected layer
        hidden_2=tf.nn.relu(tf.matmul(hidden_drop, layer4_weights)+layer4_biases)
        hidden_2_drop=tf.nn.dropout(hidden_2,keep_prob)
        
        return tf.matmul(hidden_2_drop, layer5_weights)+ layer5_biases
    
    #training computation
    
    logits=model(tf_train_dataset)
    loss=tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels,logits=logits))
    
    #optimizer
    global_step=tf.Variable(0)
    start_learning_rate=0.05
    learning_rate=tf.train.exponential_decay(start_learning_rate, global_step,100000,0.96,
                                             staircase=True)
    
    optimizer=tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
    
    #Predictions for the training , validation, and test data
    
    train_prediction= tf.nn.softmax(logits)
    valid_prediction= tf.nn.softmax(model(tf_valid_dataset))
    test_prediction= tf.nn.softmax(model(tf_test_dataset))

In [ ]:
num_steps=20000

with tf.Session(graph=graph)as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        offset=(step*batch_size)% (train_labels.shape[0]-batch_size)
        batch_data=train_dataset[offset:(offset+batch_size),:,:,:]
        batch_labels=train_labels[offset:(offset+batch_size),:]
        feed_dict={tf_train_dataset : batch_data,tf_train_labels: batch_labels}
        _, l, predictions =session.run([optimizer,loss,train_prediction],feed_dict=feed_dict)
        if(step%50==0):
            print('Minibatch loss a step %d: %f' %(step,l))
            print('Minibatch Accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%'%accuracy(valid_prediction.eval(),valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss a step 0: 3.780119
Minibatch Accuracy: 6.2%
Validation accuracy: 9.8%
Minibatch loss a step 50: 2.171072
Minibatch Accuracy: 31.2%
Validation accuracy: 18.9%
Minibatch loss a step 100: 2.094043
Minibatch Accuracy: 18.8%
Validation accuracy: 26.3%
Minibatch loss a step 150: 1.615469
Minibatch Accuracy: 43.8%
Validation accuracy: 34.4%
Minibatch loss a step 200: 1.728094
Minibatch Accuracy: 37.5%
Validation accuracy: 38.7%
Minibatch loss a step 250: 1.742614
Minibatch Accuracy: 31.2%
Validation accuracy: 43.0%
Minibatch loss a step 300: 1.413950
Minibatch Accuracy: 56.2%
Validation accuracy: 46.7%
Minibatch loss a step 350: 1.089064
Minibatch Accuracy: 62.5%
Validation accuracy: 48.1%
Minibatch loss a step 400: 1.185003
Minibatch Accuracy: 68.8%
Validation accuracy: 50.5%
Minibatch loss a step 450: 1.873650
Minibatch Accuracy: 37.5%
Validation accuracy: 49.9%
Minibatch loss a step 500: 1.279022
Minibatch Accuracy: 68.8%
Validation accuracy: 54.8%
Minibatch loss